In [1]:
from PIL import Image
import numpy as np
from scipy import ndimage
import os
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.io import read_image
import random


frame_size = 640
img_ratio = 0.075/0.1 # resolutions of given dataset and Google - (156543.03392 * math.cos(math.radians(lat))) / (2 ** zoom)

input_dir = '/home/ondin/Developer/FVAPP/data_big/archive/train/'
output_dir = '/home/ondin/Developer/FVAPP/data_big/archive/train/'


In [2]:
img_path = '/home/ondin/Developer/FVAPP/data_big/archive/train/image_resized/christchurch_309_1.png'
label_path = '/home/ondin/Developer/FVAPP/data_big/archive/train/label_resized/christchurch_309_1.png'

In [10]:
im = np.array(Image.open(img_path))
l = np.array(Image.open(label_path)).astype('float')

In [3]:
ii = read_image(img_path)

In [4]:
ii.shape

torch.Size([3, 640, 640])

In [11]:
l = T.ToTensor()(l)
l.int()

tensor([[[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.int32)

tensor([[[0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)

In [59]:

def random_rot90(image):
    k = random.randint(0, 3)
    return torch.rot90(image, k=k, dims=(1, 2))


training_augmentation = T.Compose([
    T.ToTensor(),
    T.Lambda(random_rot90)
    ]
)

In [64]:
torch.stack([l==i for i in range(2)], dim=1).sum(dim=(2))

torch.Size([1, 2, 640])

In [52]:
aim = training_augmentation(im)
aim = aim.permute(1, 2, 0).numpy()

In [94]:
al = training_augmentation(l)

In [95]:
al.shape

torch.Size([1, 640, 640])

In [57]:
im.shape

(640, 640, 3)

In [56]:
T.ToTensor()(im).shape

torch.Size([3, 640, 640])

In [3]:
imgs_path = os.path.join(input_dir, 'image')
labels_path = os.path.join(input_dir, 'label')

out_imgs_path = os.path.join(output_dir, 'image_resized')
out_labels_path = os.path.join(output_dir, 'label_resized')

if not os.path.exists(out_imgs_path):
    os.makedirs(out_imgs_path)
if not os.path.exists(out_labels_path):
    os.makedirs(out_labels_path)

In [5]:
img_names = os.listdir(imgs_path)
for img_name in img_names:
    img_path = os.path.join(imgs_path, img_name)
    label_path = os.path.join(labels_path, img_name)

    image = np.array(Image.open(img_path))
    label = np.array(Image.open(label_path))

    new_shape = (int(label.shape[0] * img_ratio), int(label.shape[1] * img_ratio))
    label = cv2.resize(label, dsize=new_shape, interpolation=cv2.INTER_AREA)
    image = cv2.resize(image, dsize=new_shape, interpolation=cv2.INTER_AREA)

    data = label
    # Find the positive label areas
    positive_label_areas = np.zeros((data.shape[0], data.shape[1]), dtype=np.uint8)
    positive_label_areas[np.where(data == 1)] = 1

    # Generate a label mask that assigns a unique label to each positive area
    label_mask, num_labels = ndimage.label(positive_label_areas)

    # Loop through each positive label area and generate a frame
    for i in range(1, num_labels+1):
        # Find the bounding box of the positive label area
        rows, cols = np.where(label_mask == i)
        row_min, row_max = np.min(rows), np.max(rows)
        col_min, col_max = np.min(cols), np.max(cols)

        # Calculate the center of the positive label area
        center_row = (row_min + row_max) // 2
        center_col = (col_min + col_max) // 2

        # Calculate the frame boundaries

        x_min, x_max= (0, frame_size) if center_col - frame_size // 2 < 0 else (data.shape[1]-frame_size, data.shape[1]) if center_col + frame_size // 2 > data.shape[1] else (center_col - frame_size // 2, center_col + frame_size // 2)
        y_min, y_max= (0, frame_size) if center_row - frame_size // 2 < 0 else (data.shape[0]-frame_size, data.shape[0]) if center_row + frame_size // 2 > data.shape[0] else (center_row - frame_size // 2, center_row + frame_size // 2)

        # Extract the content
        img_content = image[y_min:y_max, x_min:x_max]
        label_content = label[y_min:y_max, x_min:x_max]

        # Save the frame content as a new image file
        img_out_path = os.path.join(out_imgs_path, f'{img_name[:-4]}_{i}.png')
        Image.fromarray(img_content).save(img_out_path)
        
        label_out_path = os.path.join(out_labels_path, f'{img_name[:-4]}_{i}.png')
        Image.fromarray(label_content).save(label_out_path)
        print(f'{img_name[:-4]}_{i}.png')

christchurch_309_1.png
christchurch_309_2.png
christchurch_309_3.png
christchurch_309_4.png
christchurch_309_5.png
christchurch_309_6.png
christchurch_309_7.png
christchurch_309_8.png
christchurch_309_9.png
christchurch_309_10.png
christchurch_309_11.png
christchurch_309_12.png
christchurch_309_13.png
christchurch_309_14.png
christchurch_309_15.png
christchurch_309_16.png
christchurch_309_17.png
christchurch_309_18.png
christchurch_309_19.png
christchurch_309_20.png
christchurch_309_21.png
christchurch_309_22.png
christchurch_309_23.png
christchurch_309_24.png
christchurch_309_25.png
christchurch_309_26.png
christchurch_309_27.png
christchurch_309_28.png
christchurch_309_29.png
christchurch_309_30.png
christchurch_309_31.png
christchurch_309_32.png
christchurch_309_33.png
christchurch_309_34.png
christchurch_309_35.png
christchurch_309_36.png
christchurch_309_37.png
christchurch_309_38.png
christchurch_309_39.png
christchurch_309_40.png
christchurch_309_41.png
christchurch_309_42.png
c

/home/ondin/anaconda3/envs/fva/lib/python3.9/site-packages/PIL/Image.py:3167: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


christchurch_637_1.png
christchurch_637_2.png
christchurch_637_3.png
christchurch_637_4.png
christchurch_637_5.png
christchurch_637_6.png
christchurch_637_7.png
christchurch_637_8.png
christchurch_637_9.png
christchurch_637_10.png
christchurch_637_11.png
christchurch_637_12.png
christchurch_637_13.png
christchurch_637_14.png
christchurch_637_15.png
christchurch_637_16.png
christchurch_637_17.png
christchurch_637_18.png
christchurch_637_19.png
christchurch_637_20.png
christchurch_637_21.png
christchurch_637_22.png
christchurch_637_23.png
christchurch_637_24.png
christchurch_637_25.png
christchurch_637_26.png
christchurch_637_27.png
christchurch_637_28.png
christchurch_637_29.png
christchurch_637_30.png
christchurch_637_31.png
christchurch_637_32.png
